In [1]:
from selenium import webdriver
import time
import csv
from math import ceil
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd

In [2]:
# this class is used to scrape the dataset
class Downloader_SR():
    def __init__(self, link = 'https://www.sreality.cz/hledani', lease = 'pronajem', typ = 'byty', region = 'praha'):
        #we have to define variable on sreality to define the desired dataset
        #There is couple of variables to explain how the scrape can differ
        #For example we could change lease= "prodej"
        self.link = link + '/' + lease+ '/' + typ +'/' + region + '?strana='
        self.n_matches = self.number_pg() #this fetches automaticly the number of matches in our criteria
        self.n_pages = ceil(self.n_matches/20) #then it calculates number of pages on which the links are distributed
        self.link_set = [self.link + str(k+1) for k in range(0, self.n_pages)] #with the above information we generate links to access the pages
        self.link_container = [] # here we will store links of individual properties
        
    def number_pg(self):
        #this method automaticly fetches number of matches in our criteria
        driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
        driver.get(self.link)
        time.sleep(2)
        res1 = driver.execute_script("return document.documentElement.outerHTML")
        soup = BeautifulSoup(res1, "lxml")
        pg_n = soup.find_all("span", {"class":"numero ng-binding"})
        driver.quit()
        text = pg_n[1].get_text()
        return int(text.replace("\xa0",""))
    
    @staticmethod
    def create_csv():
        #this method saves links of individual properties into csv
        #this allows for later scrape or verification of the links with the dataset
        dat = datetime.datetime.now()
        with open(f'links{dat.day}.{dat.month}.{dat.year}.csv', 'w') as f:
            writer = csv.writer(f, delimiter = ";", lineterminator = '\n')
            writer.writerow([f"These link were scraped on {dat.day}.{dat.month}.{dat.year} after {dat.hour}:{dat.minute}"])
    
    def find_a(self, limit = 1):
        # this method fetches links of individual properties
        #
        link_base = "https://www.sreality.cz"
        self.create_csv()
        driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
        dat = datetime.datetime.now()
        start_time = time.time()
        for page in self.link_set[:limit]:
            driver.get(page)
            time.sleep(2)
            res1 = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(res1, "lxml")
            links = soup.find_all("a", {"class":"title"})
            for non_sponsored_links in links[1:]:
                self.link_container.append(link_base + non_sponsored_links['href'])
            run_time = time.time() - start_time
            print(f"{len(self.link_container)} links fetched;{round((len(self.link_container)/self.n_matches)*100,3)}% finished;{round(run_time/60,2)} time taken")
            with open(f'links{dat.day}.{dat.month}.{dat.year}.csv', 'a+') as f:
                writer = csv.writer(f, delimiter = ";", lineterminator = '\n')
                for non_sponsored_links in links[1:]:
                    writer.writerow([link_base + non_sponsored_links['href']])
                    
            run_time2 = time.time() - start_time
            print("appended into csv")
            print(f"run time is {round(run_time2/60,2)} minutes")
        driver.quit()
        
    def dataF(self):
        #this method gathers the final data
        starto_time = time.time()
        box = []
        driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
        for link in self.link_container:
            driver.get(link)
            time.sleep(2)
            gist = driver.execute_script("return document.documentElement.outerHTML")
            #As sreality is constantly updated, it may happen that the links stop existing before we get to scrape it
            #This exeption handles it
            try:
                #here an object property is created and appended to list box
                pro = Property(gist)
                box.append(pro.data) # we only append a variable data(It is a pandas DataFrame) of the object pro
            except:
                print("a link cannot be accessed")
            
            elapsed_time = time.time() - starto_time
            print(f"Property has been considered {link[49:]} ; {round((len(box)/len(self.link_container))*100,1)}% finished; {round(elapsed_time/60,2)} minutes elapsed")
        driver.quit()
        #after objects are created for all individual properties, they are appended into an DataFrame
        self.df = pd.concat(box,axis = 0, sort = True)

In [3]:
class Property():
    #this an object that is created for each individual property and it is used in method dataF of class Downloader SR
    def __init__(self, soupy = " "):
        self.items = {}
        #this class prefers to get html data
         '''
         we could only receive a link and fetch the html data in this object, however it was more time demanding
         due to the fact that we are scraping dynamic webpages and we use selenium.
         If we get the html data in this object, we would have to always open and close the webdriver and 
         it was not possible due to time concerns
         '''
        try:
            self.parser(soupy)
            self.data = self.cr_data()    
        except:
            #it allows for no html data to be receive in order to test it by method scraoe
            print("no suitable html data were given")
    def parser(self, gist):
        #this method finds the variable in the parsed html
        info = BeautifulSoup(gist, "lxml")
        lists = info.find_all("li")
        self.items["Title"] = info.find("span",{"itemprop":"name"}).findChild().get_text().replace("\n","").replace("\xa0"," ")
        self.items["Adresa"] = info.find("span",{"class": "location"}).get_text().replace("\n","").replace("\xa0"," ")
        for k in lists:
            try:
                column = k.find("label").get_text()
            except:
                break
            try:
                value = [k.find("strong").get_text()]
            except:
                try:
                    value = [k.find("a").get_text(), k.find("span", {"class":"c-pois__distance ng-binding"}).get_text()]
                except:
                    value = [k.find("span",{"class":"c-pois__poi-text ng-binding ng-scope"}).get_text(),k.find("span", {"class":"c-pois__distance ng-binding"}).get_text()]

            if value[0] == "\n\n\n\n\n\n\n":
                try:
                    value[0] = k.find("strong").findChild("span",{"class": "icof icon-cross ng-scope"})["ng-if"].split(" ")[-1]
                except:
                    value[0] = k.find("strong").findChild("span",{"class": "icof icon-ok ng-scope"})["ng-if"].split(" ")[-1]

            #we clean the values a little bit
            keys = [val.replace("\n","") for val in value]
            keys_a = [ki.replace("\xa0"," ") for ki in keys]    

            #add it to the dictionary
            self.items[column] = keys_a
            
    def cr_data(self):
        #after the variables are found, this method writes them into a dataframe in a desired format
        vals = list(self.items.values())
        keys = list(self.items.keys())
        df= pd.DataFrame(zip(vals)).transpose()
        df.columns = keys
        return df
    
    
    def scrape(self, link):
        # this method is used for testing 
        driver = webdriver.Chrome("C:\\chromedriver\\chromedriver.exe")
        driver.get(link)
        res1 = driver.execute_script("return document.documentElement.outerHTML")
        self.parser(res1)
        self.data = self.cr_data()

# Code is run

an object is create of class Downloade_SR() and the dataset is obtained

In [6]:
%%time
first = Downloader_SR()

Wall time: 13.3 s


In [7]:
first.n_matches

7952

In [8]:
first.n_pages

398

In [9]:
first.link_set[-1]

'https://www.sreality.cz/hledani/pronajem/byty/praha?strana=398'

In [10]:
first.find_a(limit = first.n_pages+1)

20 links fetched;0.252% finished;0.1 time taken
appended into csv
run time is 0.1 minutes
40 links fetched;0.503% finished;0.15 time taken
appended into csv
run time is 0.15 minutes
60 links fetched;0.755% finished;0.19 time taken
appended into csv
run time is 0.19 minutes
80 links fetched;1.006% finished;0.24 time taken
appended into csv
run time is 0.24 minutes
100 links fetched;1.258% finished;0.28 time taken
appended into csv
run time is 0.28 minutes
120 links fetched;1.509% finished;0.33 time taken
appended into csv
run time is 0.33 minutes
140 links fetched;1.761% finished;0.37 time taken
appended into csv
run time is 0.37 minutes
160 links fetched;2.012% finished;0.42 time taken
appended into csv
run time is 0.42 minutes
180 links fetched;2.264% finished;0.46 time taken
appended into csv
run time is 0.46 minutes
200 links fetched;2.515% finished;0.51 time taken
appended into csv
run time is 0.51 minutes
220 links fetched;2.767% finished;0.57 time taken
appended into csv
run time

1780 links fetched;22.384% finished;4.18 time taken
appended into csv
run time is 4.18 minutes
1800 links fetched;22.636% finished;4.23 time taken
appended into csv
run time is 4.23 minutes
1820 links fetched;22.887% finished;4.32 time taken
appended into csv
run time is 4.32 minutes
1840 links fetched;23.139% finished;4.36 time taken
appended into csv
run time is 4.36 minutes
1860 links fetched;23.39% finished;4.41 time taken
appended into csv
run time is 4.41 minutes
1880 links fetched;23.642% finished;4.45 time taken
appended into csv
run time is 4.45 minutes
1900 links fetched;23.893% finished;4.49 time taken
appended into csv
run time is 4.49 minutes
1920 links fetched;24.145% finished;4.54 time taken
appended into csv
run time is 4.54 minutes
1940 links fetched;24.396% finished;4.58 time taken
appended into csv
run time is 4.58 minutes
1960 links fetched;24.648% finished;4.62 time taken
appended into csv
run time is 4.62 minutes
1980 links fetched;24.899% finished;4.67 time taken

3520 links fetched;44.266% finished;8.09 time taken
appended into csv
run time is 8.09 minutes
3540 links fetched;44.517% finished;8.13 time taken
appended into csv
run time is 8.13 minutes
3560 links fetched;44.769% finished;8.18 time taken
appended into csv
run time is 8.18 minutes
3580 links fetched;45.02% finished;8.23 time taken
appended into csv
run time is 8.23 minutes
3600 links fetched;45.272% finished;8.27 time taken
appended into csv
run time is 8.27 minutes
3620 links fetched;45.523% finished;8.32 time taken
appended into csv
run time is 8.32 minutes
3640 links fetched;45.775% finished;8.36 time taken
appended into csv
run time is 8.36 minutes
3660 links fetched;46.026% finished;8.4 time taken
appended into csv
run time is 8.4 minutes
3680 links fetched;46.278% finished;8.46 time taken
appended into csv
run time is 8.46 minutes
3700 links fetched;46.529% finished;8.52 time taken
appended into csv
run time is 8.52 minutes
3720 links fetched;46.781% finished;8.57 time taken
a

5240 links fetched;65.895% finished;12.07 time taken
appended into csv
run time is 12.07 minutes
5260 links fetched;66.147% finished;12.12 time taken
appended into csv
run time is 12.12 minutes
5280 links fetched;66.398% finished;12.16 time taken
appended into csv
run time is 12.16 minutes
5300 links fetched;66.65% finished;12.21 time taken
appended into csv
run time is 12.21 minutes
5320 links fetched;66.901% finished;12.25 time taken
appended into csv
run time is 12.25 minutes
5340 links fetched;67.153% finished;12.3 time taken
appended into csv
run time is 12.3 minutes
5360 links fetched;67.404% finished;12.35 time taken
appended into csv
run time is 12.35 minutes
5380 links fetched;67.656% finished;12.41 time taken
appended into csv
run time is 12.41 minutes
5400 links fetched;67.907% finished;12.46 time taken
appended into csv
run time is 12.46 minutes
5420 links fetched;68.159% finished;12.5 time taken
appended into csv
run time is 12.5 minutes
5440 links fetched;68.41% finished;

6940 links fetched;87.274% finished;16.37 time taken
appended into csv
run time is 16.37 minutes
6960 links fetched;87.525% finished;16.41 time taken
appended into csv
run time is 16.41 minutes
6980 links fetched;87.777% finished;16.46 time taken
appended into csv
run time is 16.46 minutes
7000 links fetched;88.028% finished;16.5 time taken
appended into csv
run time is 16.5 minutes
7020 links fetched;88.28% finished;16.55 time taken
appended into csv
run time is 16.55 minutes
7040 links fetched;88.531% finished;16.6 time taken
appended into csv
run time is 16.6 minutes
7060 links fetched;88.783% finished;16.65 time taken
appended into csv
run time is 16.65 minutes
7080 links fetched;89.034% finished;16.7 time taken
appended into csv
run time is 16.7 minutes
7100 links fetched;89.286% finished;16.74 time taken
appended into csv
run time is 16.74 minutes
7120 links fetched;89.537% finished;16.79 time taken
appended into csv
run time is 16.79 minutes
7140 links fetched;89.789% finished;1

In [11]:
first.link_container[0]

'https://www.sreality.cz/detail/pronajem/byt/1+kk/praha-cerny-most-dobrovolneho/4011146844'

In [12]:
len(first.link_container)

7960

In [13]:
%%time
first.dataF()

Property has been considered praha-cerny-most-dobrovolneho/4011146844 ; 0.0% finished; 0.63 minutes elapsed
Property has been considered praha-zizkov-borivojova/547958364 ; 0.0% finished; 0.69 minutes elapsed
Property has been considered raha-letnany-tupolevova/1950953052 ; 0.0% finished; 0.77 minutes elapsed
Property has been considered praha-smichov-podbelohorska/851131996 ; 0.1% finished; 0.86 minutes elapsed
Property has been considered raha-pisnice-mezi-domy/3857464924 ; 0.1% finished; 0.96 minutes elapsed
Property has been considered praha-pisnice-ke-kurtum/2870754908 ; 0.1% finished; 1.05 minutes elapsed
Property has been considered praha-mala-strana-snemovni/1236483676 ; 0.1% finished; 1.15 minutes elapsed
Property has been considered raha-vysocany-kolbenova/3273272924 ; 0.1% finished; 1.24 minutes elapsed
Property has been considered praha-podoli-prava/1775451740 ; 0.1% finished; 1.35 minutes elapsed
Property has been considered raha-vinohrady-rimska/3292212828 ; 0.1% finished

Property has been considered praha-radotin-tyrovicka/2293771868 ; 1.0% finished; 5.96 minutes elapsed
Property has been considered praha-letnany-frydecka/3019451996 ; 1.0% finished; 6.0 minutes elapsed
Property has been considered praha-kyje-davidovicova/3419090524 ; 1.0% finished; 6.08 minutes elapsed
Property has been considered praha-vrsovice-28--pluku/1640156764 ; 1.1% finished; 6.17 minutes elapsed
Property has been considered raha-smichov-kovaku/872693340 ; 1.1% finished; 6.25 minutes elapsed
Property has been considered praha-nusle-taborska/1050754652 ; 1.1% finished; 6.3 minutes elapsed
Property has been considered raha-zizkov-konevova/2277060188 ; 1.1% finished; 6.35 minutes elapsed
Property has been considered praha-michle-mendiku/4106825308 ; 1.1% finished; 6.73 minutes elapsed
Property has been considered praha-hloubetin-saarinenova/649084508 ; 1.1% finished; 6.78 minutes elapsed
Property has been considered praha-chodov-jazlovicka/1015496284 ; 1.1% finished; 6.82 minutes e

Property has been considered praha-podoli-vodarenska/3804642908 ; 2.0% finished; 10.62 minutes elapsed
Property has been considered praha-vinohrady-polska/565849692 ; 2.0% finished; 10.67 minutes elapsed
Property has been considered praha-chodov-na-sadce/3504877148 ; 2.0% finished; 10.71 minutes elapsed
Property has been considered praha-jinonice-klikata/3124481628 ; 2.0% finished; 10.8 minutes elapsed
Property has been considered praha-michle-michelska/3065790044 ; 2.1% finished; 10.85 minutes elapsed
Property has been considered praha-dejvice-/159370844 ; 2.1% finished; 10.89 minutes elapsed
Property has been considered raha-nusle-marie-cibulkove/495595100 ; 2.1% finished; 10.94 minutes elapsed
Property has been considered praha-mala-strana-mostecka/1202204252 ; 2.1% finished; 10.98 minutes elapsed
Property has been considered praha-nove-mesto-krakovska/1307000412 ; 2.1% finished; 11.03 minutes elapsed
Property has been considered praha-brevnov-8--listopadu/3004706396 ; 2.1% finished

Property has been considered raha-nove-mesto-barvirska/3137355356 ; 3.0% finished; 15.68 minutes elapsed
Property has been considered praha-stare-mesto-dlouha/3573956188 ; 3.0% finished; 15.73 minutes elapsed
Property has been considered praha-zizkov-vlkova/2619620956 ; 3.0% finished; 15.77 minutes elapsed
Property has been considered raha-zizkov-vlkova/4202311260 ; 3.1% finished; 15.81 minutes elapsed
Property has been considered raha-bubenec-v--p--ckalova/4006465116 ; 3.1% finished; 15.86 minutes elapsed
Property has been considered praha-smichov-pod-barvirkou/1404579420 ; 3.1% finished; 15.95 minutes elapsed
Property has been considered raha-letnany-dudkova/3015192156 ; 3.1% finished; 16.0 minutes elapsed
Property has been considered praha-pisnice-svihovska/846736988 ; 3.1% finished; 16.08 minutes elapsed
Property has been considered praha-kobylisy-chotovicka/817376860 ; 3.1% finished; 16.17 minutes elapsed
Property has been considered praha-holesovice-u-uranie/2436378204 ; 3.1% fin

Property has been considered raha-miskovice-bendlova/1542991452 ; 4.0% finished; 21.63 minutes elapsed
Property has been considered praha-ujezd-nad-lesy-starokolinska/3465035356 ; 4.0% finished; 21.68 minutes elapsed
Property has been considered praha-vinohrady-slavikova/393297500 ; 4.0% finished; 21.72 minutes elapsed
Property has been considered praha-vinohrady-polska/1147551324 ; 4.1% finished; 21.81 minutes elapsed
Property has been considered praha-modrany-mezi-vodami/1362247260 ; 4.1% finished; 21.86 minutes elapsed
Property has been considered praha-krc-za-zelenou-liskou/2154184284 ; 4.1% finished; 21.9 minutes elapsed
Property has been considered praha-strasnice-secska/3561500252 ; 4.1% finished; 21.95 minutes elapsed
Property has been considered praha-stare-mesto-naprstkova/3097046620 ; 4.1% finished; 21.99 minutes elapsed
Property has been considered praha-dejvice-lindleyova/3793174108 ; 4.1% finished; 22.08 minutes elapsed
Property has been considered praha-jinonice-kacirkov

...

Property has been considered praha-smichov-karla-englise/2466434652 ; 94.5% finished; 399.77 minutes elapsed
Property has been considered praha-zizkov-pitterova/3338849884 ; 94.5% finished; 399.84 minutes elapsed
Property has been considered praha-zizkov-jana-zelivskeho/2593406556 ; 94.5% finished; 399.9 minutes elapsed
Property has been considered praha-troja-povltavska/1653866076 ; 94.5% finished; 399.99 minutes elapsed
Property has been considered praha-vinohrady-laubova/3067846236 ; 94.6% finished; 400.1 minutes elapsed
Property has been considered raha-stare-mesto-v-kolkovne/1046474332 ; 94.6% finished; 400.15 minutes elapsed
Property has been considered raha-vinohrady-nitranska/3102797404 ; 94.6% finished; 400.2 minutes elapsed
Property has been considered praha-smichov-horejsi-nabrezi/1262669404 ; 94.6% finished; 400.27 minutes elapsed
Property has been considered praha-hloubetin-k-nahonu/2118307420 ; 94.6% finished; 400.31 minutes elapsed
Property has been considered praha-podo

Property has been considered praha-vokovice-nobelova/2174668380 ; 95.5% finished; 404.67 minutes elapsed
Property has been considered praha--malesicka/899599964 ; 95.5% finished; 404.72 minutes elapsed
Property has been considered praha-nove-mesto-na-zderaze/1961090652 ; 95.5% finished; 404.77 minutes elapsed
Property has been considered praha-michle-u-plynarny/3997494876 ; 95.5% finished; 404.82 minutes elapsed
Property has been considered praha-vinohrady-moravska/2561592924 ; 95.5% finished; 404.86 minutes elapsed
Property has been considered praha-vinohrady-lucemburska/3336490588 ; 95.5% finished; 404.91 minutes elapsed
Property has been considered raha-nove-mesto-karlovo-namesti/2121846364 ; 95.5% finished; 404.96 minutes elapsed
Property has been considered raha-dejvice-na-micance/1495191132 ; 95.6% finished; 405.01 minutes elapsed
Property has been considered praha-prosek-okrinecka/1159646812 ; 95.6% finished; 405.06 minutes elapsed
Property has been considered praha-stodulky-5--

Property has been considered praha-jinonice-klikata/3594178140 ; 96.4% finished; 408.55 minutes elapsed
Property has been considered praha-nove-mesto-resslova/3764047452 ; 96.5% finished; 408.6 minutes elapsed
Property has been considered praha-nove-mesto-petrske-namesti/1117507164 ; 96.5% finished; 408.65 minutes elapsed
Property has been considered cky/praha-nove-mesto-narodni/2294009436 ; 96.5% finished; 408.7 minutes elapsed
Property has been considered raha-nove-mesto-narodni/968609372 ; 96.5% finished; 408.75 minutes elapsed
Property has been considered praha-bubenec-mlynska/1454100060 ; 96.5% finished; 408.8 minutes elapsed
Property has been considered raha-josefov-elisky-krasnohorske/2745884252 ; 96.5% finished; 408.85 minutes elapsed
Property has been considered praha-jinonice-klikata/634113628 ; 96.5% finished; 408.9 minutes elapsed
Property has been considered praha-vinohrady-cermakova/3570126428 ; 96.5% finished; 408.95 minutes elapsed
Property has been considered praha-nov

Property has been considered praha-nove-mesto-na-struze/3951222364 ; 97.4% finished; 412.46 minutes elapsed
Property has been considered praha-nove-mesto-truhlarska/2400378460 ; 97.4% finished; 412.51 minutes elapsed
Property has been considered praha--italska/1007869532 ; 97.4% finished; 412.56 minutes elapsed
Property has been considered raha-josefov-siroka/3976388188 ; 97.5% finished; 412.61 minutes elapsed
Property has been considered praha-chodov-bachova/2969755228 ; 97.5% finished; 412.66 minutes elapsed
Property has been considered praha-chodov-bachova/1345511004 ; 97.5% finished; 412.71 minutes elapsed
Property has been considered raha-josefov-elisky-krasnohorske/770891356 ; 97.5% finished; 412.76 minutes elapsed
Property has been considered raha-josefov-elisky-krasnohorske/3813097052 ; 97.5% finished; 412.81 minutes elapsed
Property has been considered praha-josefov-/930340444 ; 97.5% finished; 412.86 minutes elapsed
Property has been considered praha-nusle-spytihnevova/249114

Property has been considered praha-nove-mesto-myslikova/112651868 ; 98.4% finished; 416.53 minutes elapsed
Property has been considered praha-nove-mesto-myslikova/3082219100 ; 98.4% finished; 416.58 minutes elapsed
Property has been considered praha-nove-mesto-myslikova/2309418588 ; 98.4% finished; 416.64 minutes elapsed
Property has been considered raha-smichov-horejsi-nabrezi/4141280860 ; 98.4% finished; 416.7 minutes elapsed
Property has been considered praha-nove-mesto-myslikova/2137312860 ; 98.4% finished; 416.77 minutes elapsed
Property has been considered praha-vinohrady-na-kozacce/4109823580 ; 98.5% finished; 416.82 minutes elapsed
Property has been considered praha-vinohrady-sazavska/3304517212 ; 98.5% finished; 416.87 minutes elapsed
Property has been considered praha-vinohrady-moravska/905375324 ; 98.5% finished; 416.93 minutes elapsed
Property has been considered ice/praha-stresovice-slunna/3689344604 ; 98.5% finished; 417.02 minutes elapsed
Property has been considered pra

In [16]:
first.df.head()

,Adresa,Aktualizace:,Balkón:,Bankomat:,Bazén:,Bezbariérový:,Bus MHD:,Celková cena:,Cena:,Cukrárna:,...,Večerka:,Vlak:,Vlastnictví:,Voda:,Vybavení:,Výtah:,Výška stropu:,Zlevněno:,Škola:,Školka:
0,"Dobrovolného, Praha 9 - Černý Most Panorama",[Dnes],NaN,"[Bankomat České spořitelny, (542 m)]",NaN,NaN,"[Generála Janouška, (132 m)]",[12 000 Kč za měsíc],NaN,"[Shisha Room Dvin, (1144 m)]",...,"[Albert Supermarket, (451 m)]","[Praha-Kyje, (1445 m)]",[Osobní],NaN,NaN,NaN,NaN,NaN,"[ZUŠ Praha 9, Ratibořická, (234 m)]","[MŠ Sluníčko Praha 9 - Černý Most, (279 m)]"
0,"Bořivojova, Praha 3 - Žižkov Panorama",[Dnes],NaN,"[Bankomat České spořitelny, (122 m)]",NaN,['boolean-false'],"[Tachovské náměstí, (443 m)]","[9 500 Kč za nemovitost, + provize RK]",NaN,"[Klub JINÝ KAFE, (283 m)]",...,"[Shalamar Foods, (165 m)]","[Praha hlavní nádraží, (984 m)]",[Osobní],NaN,NaN,NaN,NaN,NaN,"[Gymnázium Karla Sladkovského Praha 3, (56 m)]","[ZŠ a MŠ Jaroslava Seiferta Praha 3, (243 m)]"
0,"Tupolevova, Praha 9 - Letňany Panorama",[Dnes],NaN,"[Bankomat České spořitelny, (236 m)]",NaN,['boolean-true'],"[Šumperská, (177 m)]",[13 900 Kč za měsíc],NaN,"[Café Infinity, (204 m)]",...,"[Penny Market, (183 m)]","[Praha-Čakovice, (1695 m)]",[Osobní],NaN,NaN,['boolean-true'],NaN,NaN,"[Základní škola Fryčovická, (160 m)]","[MŠ Havířovská, (255 m)]"
0,"Podbělohorská, Praha 5 - Smíchov Panorama",[Dnes],NaN,"[Bankomat Komerční banky, (381 m)]",NaN,['boolean-false'],"[Pod Lipkami, (30 m)]","[15 000 Kč za nemovitost, + provize RK]",NaN,"[Cukrárna Pod Marjánkou, (1333 m)]",...,"[Smíšené zboží Plzeňská\t187, (404 m)]","[Praha-Cibulka, (1229 m)]",[Osobní],[Dálkový vodovod],NaN,NaN,NaN,NaN,"[ZUŠ Na Popelce, (224 m)]","[TROIS PETITES POMMES, s.r.o., (343 m)]"
0,"Mezi domy, Praha 4 - Písnice Panorama",[Dnes],NaN,"[Bankomat Komerční banky, (780 m)]",NaN,['boolean-false'],"[U Libušské sokolovny, (275 m)]",[19 500 Kč za měsíc],NaN,"[Cukrářství Viktoria, s.r.o., (900 m)]",...,"[VIVACOMEX, spol. s r.o., (450 m)]","[Praha-Krč, (3635 m)]",[Osobní],[Dálkový vodovod],NaN,['boolean-true'],NaN,NaN,[Střední odborné učiliště potravinářské Praha ...,"[Mateřská škola Mezi Domy, (105 m)]"


In [18]:
first.df.shape

(7878, 79)

We will export the dataset into csv to save it for future analysis

In [15]:
first.df.to_csv('dataset1.csv', sep = ';')

The data set was first successfully downloaded on the 27 april 2020 between 9am and 5pm.
The number of scraped properties and link retrieves is not one to one as some of the offerings stop existing between the time we star scraping and the time we try to access the specific link.

## Testing of Object property

In [19]:
second = Property()

no suitable html data were given


In [22]:
second.scrape("https://www.sreality.cz/detail/pronajem/byt/1+kk/praha-cerny-most-dobrovolneho/4011146844")

In [23]:
second.data

,Title,Adresa,Celková cena:,Poznámka k ceně:,ID zakázky:,Aktualizace:,Stavba:,Stav objektu:,Vlastnictví:,Podlaží:,...,Lékárna:,Bankomat:,Metro:,Obchod:,Pošta:,Tram:,Školka:,Lékař:,Sportoviště:,Vlak:
0,Pronájem bytu 1+kk 26 m²,"Dobrovolného, Praha 9 - Černý Most Panorama",[12 000 Kč za měsíc],"[+ 2.500,-Kč poplatky, elektřina, provize RK]",[N05462],[Dnes],[Skeletová],[Novostavba],[Osobní],[1. podlaží z celkem 5],...,"[Lékárna U Rajské zahrady, (500 m)]","[Bankomat České spořitelny, (542 m)]","[Rajská zahrada, (525 m)]","[Kaufland, (1486 m)]","[Pošta Praha 98 - Česká pošta, s.p., (442 m)]","[Lehovec, (1136 m)]","[MŠ Sluníčko Praha 9 - Černý Most, (279 m)]","[MUDr. Zdeněk Valentík, (363 m)]","[Veřejné hřiště na míčové hry Pospíchalova, (...","[Praha-Kyje, (1445 m)]"


### Possible improvement that were not implemented

Couple of improvements could have been made to the dataF() method of the class Downloader_SR. 
<br><br>
</ul>
<li>
1) The print did not have to print if the link stop existing in the mean time of scraping.</li> 
<li>
2) The print could have calculated estimated time and number of properties scraped. </li>
<li>
3) The properties could have been appended to csv in case of error. </li>
<li>
4) There could have been intoduced a variable - to be able to scrape links in steps. </li>
</ul><br>
Although, we did not do implement these improvement because the scraping is a lenthy process. Once we scraped the data, the improvement would not have  had any affect on the result. It would just improve future experience of scraping.